# 日よけ効果係数算定ツール

- 一次目標：日よけ効果係数算定ツールのpython上での再現


## 6.3 直達日射に対する日よけ効果係数

### 6.3.1 x+側に対する係数の算定

#### 6.3.1 a) 式(15)

$$
A_{oh0+}(x,y) = \left\{
\begin{array}{ll}
0 \hspace{24pt}(z_{y+}=0)
\\
\frac{1}{2} (x_{3y+} + x_2 / 2 - x) \frac{z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}}{z_{y+} \tan | A_{ZW,j,d,t} |} (x_{3y+} + x_2 / 2 - x)
\\
\hspace{30pt} \left( \begin{array}{ll}
x_{3y+} + x_2 / 2 - x < z_{y+} \tan | A_{ZW,j,d,t} | \\
y_{1} + y_2 / 2 - y \geq \frac{z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}}{z_{y+} \tan | A_{ZW,j,d,t} |} (x_{3y+} + x_2 / 2 - x) 
\end{array} \right)
\\
\Bigl\{ (x_{3y+} + x_2 / 2 - x) - \frac{1}{2} (y_{1} + y_2 / 2 - y) \frac{z_{y+} \tan | A_{ZW,j,d,t} |}{z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}} \Bigr\} (y_{1} + y_2 / 2 - y)
\\
\hspace{30pt} \left( \begin{array}{ll}
x_{3y+} + x_2 / 2 - x > \frac{z_{y+} \tan | A_{ZW,j,d,t} |}{z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}} (y_{1} + y_2 / 2 - y) \\
y_{1} + y_2 / 2 - y < z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}
\end{array} \right)
\\
( x_{3y+} + x_2 / 2 - x - \frac{1}{2} z_{y+} \tan | A_{ZW,j,d,t} | ) z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}
\\
\hspace{30pt} \left( \begin{array}{ll}
x_{3y+} + x_2 / 2 - x \geq z_{y+} \tan | A_{ZW,j,d,t} | \\
y_{1} + y_2 / 2 - y \geq z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}
\end{array} \right)
\end{array}
\right.  \qquad (15) 
$$

In [1]:
""" 式(15) """
import numpy as np

def calc_Aoh0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs):
 
    X_th = X3yp + X2 / 2 - XX
    Y_th = Y1 + Y2 / 2 - YY
    X_th_Z = Zyp * np.tan(abs(np.radians(Azw)))  
    Y_th_Z = Zyp * np.tan(np.radians(hs)) / np.cos(np.radians(Azw))
        
    if X_th_Z == 0 or Y_th_Z <= 0:
        Aoh0p = 0    # 式(15)条件1 と 日よけが影を落とさない条件をあわせて処理
    else:
        Aoh0p = calc_Aoh0p00(X_th, Y_th, X_th_Z, Y_th_Z)
    return Aoh0p
        
def calc_Aoh0p00(X_th, Y_th, X_th_Z, Y_th_Z):
    
    if (X_th >= X_th_Z and Y_th >= Y_th_Z):
        Aoh0p00 = (X_th - X_th_Z / 2) * Y_th_Z       # 式(15)条件4
    elif Y_th * X_th_Z >= X_th * Y_th_Z:
        Aoh0p00 = X_th ** 2 * Y_th_Z / X_th_Z / 2       # 式(15)条件2
    else:
        Aoh0p00 = (X_th - Y_th / 2 * X_th_Z / Y_th_Z) * Y_th      # 式(15)条件3    
    return Aoh0p00


In [2]:
""" 式(15) Test """
# \確認.xlsx "式(15)Aoh0p"シート → \Aoh0p.csv を作成 → \TestConfig01 下に置いて読み込み → 計算

import pandas as pd
import sys
csv_input = pd.read_csv(filepath_or_buffer="./TestConfig01/Aoh0p.csv", encoding="ms932", sep=",")
if csv_input.columns[0]!="Aoh0p_case":
    sys.exit("ファイル内に貼り付けたテスト条件が違います")
for i in range(len(csv_input)):
    [case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0pA] \
        = csv_input.values[i]
    Aoh0p = calc_Aoh0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
    print('case{}: Aohop = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0p, Aoh0pA, Aoh0p - Aoh0pA ))    

case1.0: Aohop = 0, 期待値 = 0.0, 残差 = 0.0
case2.0: Aohop = 0.0775157853238327, 期待値 = 0.0775157853238327, 残差 = 0.0
case3.0: Aohop = 0.07780003190413882, 期待値 = 0.0778000319041388, 残差 = 2.7755575615628914e-17
case4.0: Aohop = 0.03672539075835095, 期待値 = 0.0367253907583509, 残差 = 4.85722573273506e-17
case5.0: Aohop = 0.03127460375376254, 期待値 = 0.0312746037537625, 残差 = 3.469446951953614e-17
case6.0: Aohop = 0.028567119585167593, 期待値 = 0.0285671195851676, 残差 = -6.938893903907228e-18
case7.0: Aohop = 0.7830463214892595, 期待値 = 0.783046321489259, 残差 = 4.440892098500626e-16
case8.0: Aohop = 0.7859177139182305, 期待値 = 0.78591771391823, 残差 = 5.551115123125783e-16
case9.0: Aohop = 0.3709913022030704, 期待値 = 0.37099130220307, 残差 = 4.440892098500626e-16
case10.0: Aohop = 0.3159287275889663, 期待値 = 0.315928727588966, 残差 = 2.7755575615628914e-16
case11.0: Aohop = 0.28857835617943045, 期待値 = 0.28857835617943, 残差 = 4.440892098500626e-16
case12.0: Aohop = 2.563941165914074, 期待値 = 2.56394116591407, 残差 = 3.99680288

#### 6.3.1 b) 式(16)

$$
A_{sf0+}(x,y) = \left\{
\begin{array}{ll}
0 \hspace{24pt}(z_{x+}=0)
\\
\frac{1}{2} (y_{1x+} + y_2 / 2 - y) \frac{z_{x+} \tan | A_{ZW,j,d,t} |}{z_{x+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}} (y_{1x+} + y_2 / 2 - y)
\\
\hspace{30pt} \left( \begin{array}{ll}
y_{1x+} + y_2 / 2 - y < z_{x+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t} \\
x_{3} + x_2 / 2 - x \geq \frac{z_{x+} \tan | A_{ZW,j,d,t} |}{z_{x+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}} (y_{1x+} + y_2 / 2 - y) 
\end{array} \right)
\\
\Bigl\{ (y_{1x+} + y_2 / 2 - y) - \frac{1}{2} (x_{3} + x_2 / 2 - x) \frac{z_{x+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}}{z_{x+} \tan | A_{ZW,j,d,t} |} \Bigr\} (x_{3} + x_2 / 2 - x)
\\
\hspace{30pt} \left( \begin{array}{ll}
y_{1x+} + y_2 / 2 - y > \frac{z_{x+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}}{z_{x+} \tan | A_{ZW,j,d,t} |} (x_{3} + x_2 / 2 - x) \\
x_{3} + x_2 / 2 - x < z_{x+} \tan | A_{ZW,j,d,t} |
\end{array} \right)
\\
( y_{1x+} + y_2 / 2 - y - \frac{1}{2} z_{x+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t} ) \; z_{x+} \tan | A_{ZW,j,d,t} |
\\
\hspace{30pt} \left( \begin{array}{ll}
y_{1x+} + y_2 / 2 - y \geq z_{x+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t} \\
x_{3} + x_2 / 2 - x \geq z_{x+} \tan | A_{ZW,j,d,t} |
\end{array} \right)
\end{array}
\right.  \qquad (16) 
$$

- コード中では、座標を入れ替えて、`calc_Aoh0p00`を叩くことで対応
  - 式$(15)$の変数 → 式$(16)$の変数
  - $x$ → $y$
  - $x_2$ → $y_2$
  - $x_{3y+}$ → $y_{1x+}$
  - $y$ → $x$
  - $y_1$ → $x_3$
  - $y_2$ → $x_2$
  - $z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}$ → $z_{x+} \tan | A_{ZW,j,d,t} |$
  - $z_{y+} \tan | A_{ZW,j,d,t} |$ → $z_{x+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}$

In [3]:
""" 式(16) """
import numpy as np

def calc_Asf0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs):
 
    X_th = Y1xp + Y2 / 2 - YY
    Y_th = X3 + X2 / 2 - XX
    X_th_Z = Zxp * np.tan(np.radians(hs)) / np.cos(np.radians(Azw))  
    Y_th_Z = Zxp * np.tan(abs(np.radians(Azw)))
       
    if X_th_Z == 0 or Y_th_Z <= 0:
        Aoh0p = 0    # 式(16)条件1 と 日よけが影を落とさない条件をあわせて処理
    else:
        Aoh0p = calc_Aoh0p00(X_th, Y_th, X_th_Z, Y_th_Z)
    return Aoh0p

In [4]:
""" 式(16) Test """
# \確認.xlsx "式(16)Asf0p"シート → \Asf0p.csv を作成 → \TestConfig01 下に置いて読み込み → 計算

import pandas as pd
import sys
csv_input = pd.read_csv(filepath_or_buffer="./TestConfig01/Asf0p.csv", encoding="ms932", sep=",")
if csv_input.columns[0]!="Asf0p_case":
    sys.exit("ファイル内に貼り付けたテスト条件が違います")
for i in range(len(csv_input)):
    [case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0pA] \
        = csv_input.values[i]
    Asf0p = calc_Asf0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
    print('case{}: Asfop = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0p, Asf0pA, Asf0p - Asf0pA ))   

case1.0: Asfop = 0, 期待値 = 0.0, 残差 = 0.0
case2.0: Asfop = 8.951440242791485, 期待値 = 8.951440242791481, 残差 = 3.552713678800501e-15
case3.0: Asfop = 8.951152168017515, 期待値 = 8.95115216801752, 残差 = -5.329070518200751e-15
case4.0: Asfop = 1.4419562661634264, 期待値 = 1.44195626616343, 残差 = -3.552713678800501e-15
case5.0: Asfop = 0.832815119938684, 期待値 = 0.8328151199386841, 残差 = -1.1102230246251565e-16
case6.0: Asfop = 0.025183973407320585, 期待値 = 0.0251839734073206, 残差 = -1.3877787807814457e-17
case7.0: Asfop = 8.236407718863639, 期待値 = 8.23640771886364, 残差 = -1.7763568394002505e-15
case8.0: Asfop = 8.233497654918239, 期待値 = 8.23349765491824, 残差 = -1.7763568394002505e-15
case9.0: Asfop = 1.4160732663325195, 期待値 = 1.41607326633252, 残差 = -4.440892098500626e-16
case10.0: Asfop = 0.820613756806761, 期待値 = 0.8206137568067611, 残差 = -1.1102230246251565e-16
case11.0: Asfop = 0.02486446136556832, 期待値 = 0.0248644613655683, 残差 = 2.0816681711721685e-17
case12.0: Asfop = 6.431528028779486, 期待値 = 6.4315280287794

#### 6.3.1 c) 式(14)

$$ \begin{align}
A_{wind,j,x+,d,t} &= (x_2 + x_3)(y_1 + y_2) - A_{oh0+}(-x_2 / 2, -y_2 / 2) - A_{sf0+}(-x_2 / 2, -y_2 / 2) \\
&- \{ (x_2 + x_3) y_1 - A_{oh0+}(-x_2 / 2, y_2 / 2) - A_{sf0+}(-x_2 / 2, y_2 / 2) \} \\
&- \{ x_3 (y_1 + y_2) - A_{oh0+}( x_2 / 2, -y_2 / 2) - A_{sf0+}( x_2 / 2, -y_2 / 2) \} \\
&+ x_3 y_1 - A_{oh0+}( x_2 / 2, y_2 / 2) - A_{sf0+}( x_2 / 2, y_2 / 2) \qquad \qquad \qquad (14) \\
\end{align} $$

In [5]:
""" 式(14) """

def calc_Axp(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs):
    Axp = ( (X2 + X3) * (Y1 + Y2) 
           - calc_Aoh0p(-X2/2, -Y2/2, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs) 
           - calc_Asf0p(-X2/2, -Y2/2, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs) )\
        - ( (X2 + X3) * Y1        
           - calc_Aoh0p(-X2/2,  Y2/2, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs) 
           - calc_Asf0p(-X2/2,  Y2/2, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs) ) \
        - ( X3 * (Y1 + Y2)        
           - calc_Aoh0p( X2/2, -Y2/2, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs) 
           - calc_Asf0p( X2/2, -Y2/2, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs) ) \
        + ( X3 * Y1               
           - calc_Aoh0p( X2/2,  Y2/2, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs) 
           - calc_Asf0p( X2/2,  Y2/2, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs) )       
    Axp = max(0, min(Axp, X2 * Y2))    #負値は0に、X2*Y2を超える場合はX2*Y2で頭打ち
    return Axp


In [6]:
""" 式(14) Test """
# \確認.xlsx "式(14)Axp"シート → \Axp.csv を作成 → \TestConfig01 下に置いて読み込み → 計算

import pandas as pd
import sys
csv_input = pd.read_csv(filepath_or_buffer="./TestConfig01/Axp.csv", encoding="ms932", sep=",")
if csv_input.columns[0]!="Axp_case":
    sys.exit("ファイル内に貼り付けたテスト条件が違います")
for i in range(len(csv_input)):
    [case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxpA] \
        = csv_input.values[i]
    Axp = calc_Axp(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
    print('case{}: Axp = {}, 期待値 = {}, 残差 = {}'.format( case, Axp, AxpA, Axp - AxpA ))

case1.0: Axp = 4.305, 期待値 = 4.305, 残差 = 0.0
case2.0: Axp = 6.661338147750939e-16, 期待値 = 6.66133814775094e-16, 残差 = -9.860761315262648e-32
case3.0: Axp = 0.5264142670414172, 期待値 = 0.526414267041419, 残差 = -1.7763568394002505e-15
case4.0: Axp = 4.305, 期待値 = 4.305, 残差 = 0.0
case5.0: Axp = 4.305, 期待値 = 4.305, 残差 = 0.0
case6.0: Axp = 4.305, 期待値 = 4.305, 残差 = 0.0
case7.0: Axp = 1.887379141862766e-15, 期待値 = 1.8873791418627697e-15, 残差 = -3.549874073494553e-30
case8.0: Axp = 0.5264142670414177, 期待値 = 0.5264142670414179, 残差 = -2.220446049250313e-16
case9.0: Axp = 4.305, 期待値 = 4.305, 残差 = 0.0
case10.0: Axp = 4.304999999999999, 期待値 = 4.305, 残差 = -8.881784197001252e-16
case11.0: Axp = 4.305, 期待値 = 4.305, 残差 = 0.0
case12.0: Axp = 0.04133413735431657, 期待値 = 0.0413341373543166, 残差 = -2.7755575615628914e-17
case13.0: Axp = 0.48453051747710896, 期待値 = 0.484530517477109, 残差 = -5.551115123125783e-17
case14.0: Axp = 4.304999999999998, 期待値 = 4.305, 残差 = -1.7763568394002505e-15
case15.0: Axp = 4.30499999999999

### 6.3.2 x-側に対する係数の算定

#### 6.3.2 a) 式(19)

$$
A_{oh0-}(x,y) = \left\{
\begin{array}{ll}
0 \hspace{24pt}(z_{y+}=0)
\\
\frac{1}{2} (x_{1y+} + x_2 / 2 + x) \frac{z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}}{z_{y+} \tan A_{ZW,j,d,t}} (x_{1y+} + x_2 / 2 + x)
\\
\hspace{30pt} \left( \begin{array}{ll}
x_{1y+} + x_2 / 2 + x < z_{y+} \tan A_{ZW,j,d,t} \\
y_{1} + y_2 / 2 - y \geq \frac{z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}}{z_{y+} \tan A_{ZW,j,d,t}} (x_{1y+} + x_2 / 2 + x) 
\end{array} \right)
\\
\Bigl\{ (x_{1y+} + x_2 / 2 + x) - \frac{1}{2} (y_{1} + y_2 / 2 - y) \frac{z_{y+} \tan A_{ZW,j,d,t}}{z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}} \Bigr\} (y_{1} + y_2 / 2 - y)
\\
\hspace{30pt} \left( \begin{array}{ll}
x_{1y+} + x_2 / 2 + x > \frac{z_{y+} \tan A_{ZW,j,d,t}}{z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}} (y_{1} + y_2 / 2 - y) \\
y_{1} + y_2 / 2 - y < z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}
\end{array} \right)
\\
( x_{1y+} + x_2 / 2 + x - \frac{1}{2} z_{y+} \tan A_{ZW,j,d,t} ) z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}
\\
\hspace{30pt} \left( \begin{array}{ll}
x_{1y+} + x_2 / 2 + x \geq z_{y+} \tan A_{ZW,j,d,t} \\
y_{1} + y_2 / 2 - y \geq z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}
\end{array} \right)
\end{array}
\right.  \qquad (19) 
$$

- コード中では、座標を入れ替えて、`calc_Aoh0p00`を叩くことで対応
  - 式$(15)$の変数 → 式$(19)$の変数
  - $x$ → $-x$
  - $x_{3y+}$ → $x_{1y+}$
  - $A_{ZW,j,d,t}$ → $-A_{ZW,j,d,t}$

In [7]:
""" 式(19) """
import numpy as np

def calc_Aoh0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs):
 
    X_th = X1yp + X2 / 2 + XX
    Y_th = Y1 + Y2 / 2 - YY
    X_th_Z = Zyp * np.tan(abs(np.radians(Azw)))  
    Y_th_Z = Zyp * np.tan(np.radians(hs)) / np.cos(np.radians(Azw))
        
    if X_th_Z == 0 or Y_th_Z <= 0:
        Aoh0m = 0    # 式(19)条件1 と 日よけが影を落とさない条件をあわせて処理
    else:
        Aoh0m = calc_Aoh0p00(X_th, Y_th, X_th_Z, Y_th_Z)
    return Aoh0m

In [8]:
""" 式(19) Test """
# \確認.xlsx "式(19)Aoh0m"シート → \Aoh0m.csv を作成 → \TestConfig01 下に置いて読み込み → 計算

import pandas as pd
import sys
csv_input = pd.read_csv(filepath_or_buffer="./TestConfig01/Aoh0m.csv", encoding="ms932", sep=",")
if csv_input.columns[0]!="Aoh0m_case":
    sys.exit("ファイル内に貼り付けたテスト条件が違います")
for i in range(len(csv_input)):
    [case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0mA] \
        = csv_input.values[i]
    Aoh0m = calc_Aoh0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
    print('case{}: Aoh0m = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0m, Aoh0mA, Aoh0m - Aoh0mA ))

case1.0: Aoh0m = 0, 期待値 = 0.0, 残差 = 0.0
case2.0: Aoh0m = 0.009623570258043054, 期待値 = 0.00962357025804305, 残差 = 3.469446951953614e-18
case3.0: Aoh0m = 0.009658859417854266, 期待値 = 0.00965885941785427, 残差 = -5.204170427930421e-18
case4.0: Aoh0m = 0.010522062047217, 期待値 = 0.010522062047217, 残差 = 0.0
case5.0: Aoh0m = 0.009879675452196347, 期待値 = 0.00987967545219635, 残差 = -3.469446951953614e-18
case6.0: Aoh0m = 0.010035745745870175, 期待値 = 0.0100357457458702, 残差 = -2.42861286636753e-17
case7.0: Aoh0m = 0.09721505443920411, 期待値 = 0.0972150544392041, 残差 = 1.3877787807814457e-17
case8.0: Aoh0m = 0.09757153727251575, 期待値 = 0.0975715372725158, 残差 = -5.551115123125783e-17
case9.0: Aoh0m = 0.10629140820975215, 期待値 = 0.106291408209752, 残差 = 1.5265566588595902e-16
case10.0: Aoh0m = 0.09980216917149039, 期待値 = 0.0998021691714904, 残差 = -1.3877787807814457e-17
case11.0: Aoh0m = 0.1013787547513757, 期待値 = 0.10137875475137599, 残差 = -2.914335439641036e-16
case12.0: Aoh0m = 0.31831281647451304, 期待値 = 0.31831281

#### 6.3.2 b) 式(20)

$$
A_{sf0-}(x, y) = \left\{
\begin{array}{ll}
0 \hspace{24pt}(z_{x-}=0)
\\
\frac{1}{2} (y_{1x-} + y_2 / 2 - y) \frac{z_{x-} \tan A_{ZW,j,d,t}}{z_{x-} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}} (y_{1x-} + y_2 / 2 - y)
\\
\hspace{30pt} \left( \begin{array}{ll}
y_{1x-} + y_2 / 2 - y < z_{x-} \tan h_{S,d,t} / \cos A_{ZW,j,d,t} \\
x_{1} + x_2 / 2 + x \geq \frac{z_{x-} \tan A_{ZW,j,d,t}}{z_{x-} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}} (y_{1x-} + y_2 / 2 - y) 
\end{array} \right)
\\
\Bigl\{ (y_{1x-} + y_2 / 2 - y) - \frac{1}{2} (x_{1} + x_2 / 2 + x) \frac{z_{x-} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}}{z_{x-} \tan A_{ZW,j,d,t} } \Bigr\} (x_{1} + x_2 / 2 + x)
\\
\hspace{30pt} \left( \begin{array}{ll}
y_{1x-} + y_2 / 2 - y > \frac{z_{x-} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}}{z_{x-} \tan A_{ZW,j,d,t}} (x_{1} + x_2 / 2 + x) \\
x_{1} + x_2 / 2 + x < z_{x-} \tan A_{ZW,j,d,t}
\end{array} \right)
\\
( y_{1x-} + y_2 / 2 - y - \frac{1}{2} z_{x-} \tan h_{S,d,t} / \cos A_{ZW,j,d,t} ) \; z_{x-} \tan A_{ZW,j,d,t}
\\
\hspace{30pt} \left( \begin{array}{ll}
y_{1x-} + y_2 / 2 - y \geq z_{x-} \tan h_{S,d,t} / \cos A_{ZW,j,d,t} \\
x_{1} + x_2 / 2 + x \geq z_{x-} \tan A_{ZW,j,d,t}
\end{array} \right)
\end{array}
\right.  \qquad (20) 
$$

- コード中では、座標を入れ替えて、`calc_Aoh0p00`を叩くことで対応
  - 式$(15)$の変数 → 式$(16)$の変数 → 式$(20)$の変数
  - $x$            → $y$            → $y$
  - $x_2$          → $y_2$          → $y_2$
  - $x_{3y+}$      → $y_{1x+}$      → $y_{1x-}$
  - $y$            → $x$            → $-x$
  - $y_1$          → $x_3$          → $x_1$
  - $y_2$          → $x_2$          → $x_2$
  - $z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}$ → $z_{x+} \tan | A_{ZW,j,d,t} |$ → $z_{x-} \tan A_{ZW,j,d,t}$
  - $z_{y+} \tan | A_{ZW,j,d,t} |$ → $z_{x+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}$ → $z_{x-} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}$

In [9]:
""" 式(20) """
import numpy as np

def calc_Asf0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs):
 
    X_th = Y1xm + Y2 / 2 - YY
    Y_th = X1 + X2 / 2 + XX
    X_th_Z = Zxm * np.tan(np.radians(hs)) / np.cos(np.radians(Azw))  
    Y_th_Z = Zxm * np.tan(abs(np.radians(Azw)))
       
    if X_th_Z == 0 or Y_th_Z <= 0:
        Asf0m = 0    # 式(20)条件1 と 日よけが影を落とさない条件をあわせて処理
    else:
        Asf0m = calc_Aoh0p00(X_th, Y_th, X_th_Z, Y_th_Z)
    return Asf0m

In [10]:
""" 式(20) Test """
# \確認.xlsx "式(20)Asf0m"シート → \Asf0m.csv を作成 → \TestConfig01 下に置いて読み込み → 計算

import pandas as pd
import sys
csv_input = pd.read_csv(filepath_or_buffer="./TestConfig01/Asf0m.csv", encoding="ms932", sep=",")
if csv_input.columns[0]!="Asf0m_case":
    sys.exit("ファイル内に貼り付けたテスト条件が違います")
for i in range(len(csv_input)):
    [case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0mA] \
        = csv_input.values[i]
    Asf0m = calc_Asf0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
    print('case{}: Asfom = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0m, Asf0mA, Asf0m - Asf0mA ))

case1.0: Asfom = 0, 期待値 = 0.0, 残差 = 0.0
case2.0: Asfom = 3.2564380770864116, 期待値 = 3.25643807708641, 残差 = 1.7763568394002505e-15
case3.0: Asfom = 3.2563993470334665, 期待値 = 3.2563993470334704, 残差 = -3.9968028886505635e-15
case4.0: Asfom = 1.5410625623723546, 期待値 = 1.5410625623723502, 残差 = 4.440892098500626e-15
case5.0: Asfom = 0.890086472550928, 期待値 = 0.890086472550928, 残差 = 0.0
case6.0: Asfom = 0.02691640336011408, 期待値 = 0.0269164033601141, 残差 = -2.0816681711721685e-17
case7.0: Asfom = 3.160305926647223, 期待値 = 3.1603059266472204, 残差 = 2.6645352591003757e-15
case8.0: Asfom = 3.159914684716786, 期待値 = 3.1599146847167905, 残差 = -4.440892098500626e-15
case9.0: Asfom = 1.510685986181915, 期待値 = 1.51068598618192, 残差 = -4.884981308350689e-15
case10.0: Asfom = 0.8757668172086017, 期待値 = 0.8757668172086019, 残差 = -2.220446049250313e-16
case11.0: Asfom = 0.026541420477779822, 期待値 = 0.0265414204777798, 残差 = 2.0816681711721685e-17
case12.0: Asfom = 2.9176498794247983, 期待値 = 2.9176498794247996, 残差 = -1.

#### 6.3.2 c) 式(18)

$$ \begin{align}
A_{wind,j,x-,d,t} &= (x_1 + x_2)(y_1 + y_2) - A_{oh0-}( x_2 / 2, -y_2 / 2) - A_{sf0-}( x_2 / 2, -y_2 / 2) \\
&- \{ (x_1 + x_2) y_1 - A_{oh0-}( x_2 / 2, y_2 / 2) - A_{sf0-}( x_2 / 2, y_2 / 2) \} \\
&- \{ x_1 (y_1 + y_2) - A_{oh0-}(-x_2 / 2, -y_2 / 2) - A_{sf0-}(-x_2 / 2, -y_2 / 2) \} \\
&+ x_1 y_1 - A_{oh0-}(-x_2 / 2, y_2 / 2) - A_{sf0-}(-x_2 / 2, y_2 / 2) \qquad \qquad \qquad (18) \\
\end{align} $$

In [11]:
""" 式(18) """

def calc_Axm(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs):
    Axm = ( (X1 + X2) * (Y1 + Y2) 
           - calc_Aoh0m( X2/2, -Y2/2, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs) 
           - calc_Asf0m( X2/2, -Y2/2, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs) )\
        - ( (X1 + X2) * Y1        
           - calc_Aoh0m( X2/2,  Y2/2, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs) 
           - calc_Asf0m( X2/2,  Y2/2, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs) ) \
        - ( X1 * (Y1 + Y2)        
           - calc_Aoh0m(-X2/2, -Y2/2, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs) 
           - calc_Asf0m(-X2/2, -Y2/2, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs) ) \
        + ( X1 * Y1               
           - calc_Aoh0m(-X2/2,  Y2/2, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs) 
           - calc_Asf0m(-X2/2,  Y2/2, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs) )       
    Axm = max(0, min(Axm, X2 * Y2))    #負値は0に、X2*Y2を超える場合はX2*Y2で頭打ち
    return Axm

In [12]:
""" 式(18) Test """
# \確認.xlsx "式(18)Axm"シート → \Axm.csv を作成 → \TestConfig01 下に置いて読み込み → 計算

import pandas as pd
import sys
csv_input = pd.read_csv(filepath_or_buffer="./TestConfig01/Axm.csv", encoding="ms932", sep=",")
if csv_input.columns[0]!="Axm_case":
    sys.exit("ファイル内に貼り付けたテスト条件が違います")
for i in range(len(csv_input)):
    [case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxmA] \
        = csv_input.values[i]
    Axm = calc_Axm(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
    print('case{}: Axm = {}, 期待値 = {}, 残差 = {}'.format( case, Axm, AxmA, Axm - AxmA ))

case1.0: Axm = 4.305, 期待値 = 4.305, 残差 = 0.0
case2.0: Axm = 6.661338147750939e-16, 期待値 = 6.66133814775094e-16, 残差 = -9.860761315262648e-32
case3.0: Axm = 0.5264142670414172, 期待値 = 0.526414267041419, 残差 = -1.7763568394002505e-15
case4.0: Axm = 4.305, 期待値 = 4.305, 残差 = 0.0
case5.0: Axm = 4.305, 期待値 = 4.305, 残差 = 0.0
case6.0: Axm = 4.305, 期待値 = 4.305, 残差 = 0.0
case7.0: Axm = 1.887379141862766e-15, 期待値 = 1.8873791418627697e-15, 残差 = -3.549874073494553e-30
case8.0: Axm = 0.5264142670414177, 期待値 = 0.5264142670414179, 残差 = -2.220446049250313e-16
case9.0: Axm = 4.305, 期待値 = 4.305, 残差 = 0.0
case10.0: Axm = 4.304999999999999, 期待値 = 4.305, 残差 = -8.881784197001252e-16
case11.0: Axm = 4.305, 期待値 = 4.305, 残差 = 0.0
case12.0: Axm = 0.04133413735431657, 期待値 = 0.0413341373543166, 残差 = -2.7755575615628914e-17
case13.0: Axm = 0.48453051747710896, 期待値 = 0.484530517477109, 残差 = -5.551115123125783e-17
case14.0: Axm = 4.304999999999998, 期待値 = 4.305, 残差 = -1.7763568394002505e-15
case15.0: Axm = 4.30499999999999

## 6.4 天空日射に対する日よけ効果係数


#### 6.4 a) 式(23)

$$ \begin{align}
f_{A}(x_a,x_b,y_a,y_b,z_a) &= \frac{x_b \sqrt{y_b^2+z_a^2}}{2} \tan^{-1} \frac{x_b}{\sqrt{y_b^2+z_a^2}}
- \frac{x_b \sqrt{y_a^2+z_a^2}}{2} \tan^{-1} \frac{x_b}{\sqrt{y_a^2+z_a^2}} \\
&- \frac{x_a \sqrt{y_b^2+z_a^2}}{2} \tan^{-1} \frac{x_a}{\sqrt{y_b^2+z_a^2}}
+ \frac{x_a \sqrt{y_a^2+z_a^2}}{2} \tan^{-1} \frac{x_a}{\sqrt{y_a^2+z_a^2}} \\
&+ \frac{x_b^2 - y_b^2 - z_a^2}{8} \log (x_b^2 + y_b^2 + z_a^2) - \frac{x_b^2 - y_a^2 - z_a^2}{8} \log (x_b^2 + y_a^2 + z_a^2) \\
&- \frac{x_a^2 - y_b^2 - z_a^2}{8} \log (x_a^2 + y_b^2 + z_a^2) + \frac{x_a^2 - y_a^2 - z_a^2}{8} \log (x_a^2 + y_a^2 + z_a^2) 
\qquad \qquad \qquad (23) \\
\end{align} $$

In [13]:
""" 式(23) """

def calc_fa_atan(x, y, z):
    import numpy as np

    if y**2 + z**2 > 0:
        fa_atan = x * ( y**2 + z**2 ) **0.5 / 2 * np.arctan( x / ( y**2 + z**2 ) **0.5 )
    else:
        fa_atan = 0
    
    return fa_atan

def calc_fa_log(x, y, z):
    import numpy as np
    
    if x**2 + y**2 + z**2 > 0:
        fa_log = ( x**2 - y**2 - z**2 ) / 8 * np.log( x**2 + y**2 + z**2 )
    else:
        fa_log = 0
        
    return fa_log

def calc_fa(xa, xb, ya, yb, za):
    
    fa = ( calc_fa_atan(xb, yb, za) - calc_fa_atan(xb, ya, za)
         - calc_fa_atan(xa, yb, za) + calc_fa_atan(xa, ya, za)
         + calc_fa_log(xb, yb, za) - calc_fa_log(xb, ya, za)
         - calc_fa_log(xa, yb, za) + calc_fa_log(xa, ya, za) )
  
    return fa    

In [14]:
""" 式(23) Test """
# \確認.xlsx "式(23)fA"シート → \fA.csv を作成 → \TestConfig01 下に置いて読み込み → 計算

import pandas as pd
import sys
csv_input = pd.read_csv(filepath_or_buffer="./TestConfig01/fA.csv", encoding="ms932", sep=",")
if csv_input.columns[0]!="fA_case":
    sys.exit("ファイル内に貼り付けたテスト条件が違います")
for i in range(len(csv_input)):
    [case, xa, xb, ya, yb, za, fAA] = csv_input.values[i]
    fA = calc_fa(xa, xb, ya, yb, za)
    print('case{}: fA = {}, 期待値 = {}, 残差 = {}'.format( case, fA, fAA, fA - fAA ))

case1.0: fA = 0.3926990816987242, 期待値 = 0.39269908169872403, 残差 = 1.6653345369377348e-16
case2.0: fA = 0.39269908169872403, 期待値 = 0.39269908169872403, 残差 = 0.0
case3.0: fA = 1.017540754196072, 期待値 = 1.01754075419607, 残差 = 1.9984014443252818e-15
case4.0: fA = 0.5532555723979637, 期待値 = 0.553255572397964, 残差 = -2.220446049250313e-16
case5.0: fA = 1.0175407543969328, 期待値 = 1.0175407543969301, 残差 = 2.6645352591003757e-15
case6.0: fA = 0.3910766477525088, 期待値 = 0.39107664775250894, 残差 = -1.6653345369377348e-16
case7.0: fA = 0.38630341771489735, 期待値 = 0.386303417714897, 残差 = 3.3306690738754696e-16
case8.0: fA = 0.35635255408894345, 期待値 = 0.356352554088943, 残差 = 4.440892098500626e-16
case9.0: fA = 0.2816973811292216, 期待値 = 0.281697381129222, 残差 = -3.885780586188048e-16
case10.0: fA = 0.15758009011614416, 期待値 = 0.15758009011614402, 残差 = 1.3877787807814457e-16
case11.0: fA = 0.484524567192072, 期待値 = 0.48452456719207204, 残差 = -5.551115123125783e-17
case12.0: fA = 0.8752111581280538, 期待値 = 0.87521

#### 6.4 b) 式(22)

$$ \begin{align}
\phi_{j,y+} = \{ & f_A(x_{3y+}, x_2 + x_{3y+}, y_1, y_1 + y_2, z_{y+}) + f_A(y_{1x+}, y_{1x+} + y_2, x_{3}, x_2 + x_{3}, z_{x+}) \\
+ & f_A(x_{1y+}, x_{1y+} + x_2, y_1, y_1 + y_2, z_{y+}) + f_A(y_{1x-}, y_{1x-} + y_2, x_{1}, x_1 + x_2, z_{x-}) \\ 
+ & f_A(x_{3}, x_2 + x_{3}, y_1, y_1 + y_2, 0) + f_A(y_{1}, y_{1} + y_2, x_{3}, x_2 + x_{3}, 0) \\
+ & f_A(x_{1}, x_{1} + x_2, y_1, y_1 + y_2, 0) + f_A(y_{1}, y_{1} + y_2, x_{1}, x_1 + x_2, 0) \\ 
- & f_A(x_{3y+}, x_2 + x_{3y+}, y_1, y_1 + y_2, 0) - f_A(y_{1x+}, y_{1x+} + y_2, x_{3}, x_2 + x_{3}, 0) \\
- & f_A(x_{1y+}, x_{1y+} + x_2, y_1, y_1 + y_2, 0) - f_A(y_{1x-}, y_{1x-} + y_2, x_{1}, x_1 + x_2, 0) \qquad (22) \\
\end{align} $$

In [15]:
""" 式(22) """

def calc_phiyp(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym):
    import numpy as np
    
    phiyp = ( 1 / ( np.pi * X2 * Y2 )
            * ( calc_fa(X3yp, X2 + X3yp, Y1, Y1 + Y2, Zyp) + calc_fa(Y1xp, Y1xp + Y2, X3, X2 + X3, Zxp) 
              + calc_fa(X1yp, X1yp + X2, Y1, Y1 + Y2, Zyp) + calc_fa(Y1xm, Y1xm + Y2, X1, X1 + X2, Zxm)  
              + calc_fa(X3,   X2  +  X3, Y1, Y1 + Y2, 0  ) + calc_fa(Y1,   Y1  +  Y2, X3, X2 + X3, 0  ) 
              + calc_fa(X1,   X1  +  X2, Y1, Y1 + Y2, 0  ) + calc_fa(Y1,   Y1  +  Y2, X1, X1 + X2, 0  )     
              - calc_fa(X3yp, X2 + X3yp, Y1, Y1 + Y2, 0  ) - calc_fa(Y1xp, Y1xp + Y2, X3, X2 + X3, 0  ) 
              - calc_fa(X1yp, X1yp + X2, Y1, Y1 + Y2, 0  ) - calc_fa(Y1xm, Y1xm + Y2, X1, X1 + X2, 0  ) ) ) 
    
    return phiyp     

In [16]:
""" 式(22) Test """
# \確認.xlsx "式(22)phiyp"シート → \phiyp.csv を作成 → \TestConfig01 下に置いて読み込み → 計算

import pandas as pd
import sys
csv_input = pd.read_csv(filepath_or_buffer="./TestConfig01/phiyp.csv", encoding="ms932", sep=",")
if csv_input.columns[0]!="phiyp_case":
    sys.exit("ファイル内に貼り付けたテスト条件が違います")
for i in range(len(csv_input)):
    [case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, phiypA] \
        = csv_input.values[i]
    phiyp = calc_phiyp(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym)
    print('case{}: phiyp = {}, 期待値 = {}, 残差 = {}'.format( case, phiyp, phiypA, phiyp - phiypA ))

case1.0: phiyp = 0.5000000000000002, 期待値 = 0.5, 残差 = 2.220446049250313e-16
case2.0: phiyp = 0.4985317104851181, 期待値 = 0.49853171048511796, 残差 = 1.1102230246251565e-16
case3.0: phiyp = 0.4942063220429605, 期待値 = 0.49420632204296, 残差 = 4.996003610813204e-16
case4.0: phiyp = 0.477988157781625, 期待値 = 0.477988157781625, 残差 = 0.0
case5.0: phiyp = 0.30845256398716436, 期待値 = 0.30845256398716403, 残差 = 3.3306690738754696e-16
case6.0: phiyp = 0.05934034945735664, 期待値 = 0.0593403494573566, 残差 = 4.163336342344337e-17
case7.0: phiyp = 0.5, 期待値 = 0.5, 残差 = 0.0
case8.0: phiyp = 0.45454757553749914, 期待値 = 0.45454757553749897, 残差 = 1.6653345369377348e-16
case9.0: phiyp = 0.41466359408294007, 期待値 = 0.41466359408294, 残差 = 5.551115123125783e-17
case10.0: phiyp = 0.34718878513135915, 期待値 = 0.34718878513135903, 残差 = 1.1102230246251565e-16
case11.0: phiyp = 0.13524984778837892, 期待値 = 0.135249847788379, 残差 = -8.326672684688674e-17
case12.0: phiyp = 0.015565558189019774, 期待値 = 0.0155655581890198, 残差 = -2.6020852

#### 6.4 c) 式(21)

$$\gamma_{isr,j,y+} = 2 \phi_{j,y+} \qquad (21) $$

In [17]:
""" 式(21) """

def calc_gammayp(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym):
    
    gammayp = 2 * calc_phiyp(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym)
         
    return gammayp     

## 6.5 反射日射に対する日よけ効果係数

#### 6.5 a) 式(25)

$$ \begin{align}
\phi_{j,y-} = \{ & f_A(x_{3y-}, x_2 + x_{3y-}, y_3, y_2 + y_3, z_{y-}) + f_A(y_{3x+}, y_2 + y_{3x+}, x_{3}, x_2 + x_{3}, z_{x+}) \\
+ & f_A(x_{1y-}, x_{1y-} + x_2, y_3, y_2 + y_3, z_{y-}) + f_A(y_{3x-}, y_2 + y_{3x-}, x_{1}, x_1 + x_2, z_{x-}) \\ 
+ & f_A(x_{3}, x_2 + x_{3}, y_3, y_2 + y_3, 0) + f_A(y_{3}, y_{2} + y_3, x_{3}, x_2 + x_{3}, 0) \\
+ & f_A(x_{1}, x_{1} + x_2, y_3, y_2 + y_3, 0) + f_A(y_{3}, y_{2} + y_3, x_{1}, x_1 + x_2, 0) \\ 
- & f_A(x_{3y-}, x_2 + x_{3y-}, y_3, y_2 + y_3, 0) - f_A(y_{3x+}, y_2 + y_{3x+}, x_{3}, x_2 + x_{3}, 0) \\
- & f_A(x_{1y-}, x_{1y-} + x_2, y_3, y_2 + y_3, 0) - f_A(y_{3x-}, y_2 + y_{3x-}, x_{1}, x_1 + x_2, 0) \qquad (25) \\
\end{align} $$

In [18]:
""" 式(25) """

def calc_phiym(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym):
    import numpy as np
    
    phiym = ( 1 / ( np.pi * X2 * Y2 )
            * ( calc_fa(X3ym, X2 + X3ym, Y3, Y2 + Y3, Zym) + calc_fa(Y3xp, Y2 + Y3xp, X3, X2 + X3, Zxp) 
              + calc_fa(X1ym, X1ym + X2, Y3, Y2 + Y3, Zym) + calc_fa(Y3xm, Y2 + Y3xm, X1, X1 + X2, Zxm)  
              + calc_fa(X3,   X2  +  X3, Y3, Y2 + Y3, 0  ) + calc_fa(Y3,   Y2  +  Y3, X3, X2 + X3, 0  ) 
              + calc_fa(X1,   X1  +  X2, Y3, Y2 + Y3, 0  ) + calc_fa(Y3,   Y2  +  Y3, X1, X1 + X2, 0  )     
              - calc_fa(X3ym, X2 + X3ym, Y3, Y2 + Y3, 0  ) - calc_fa(Y3xp, Y2 + Y3xp, X3, X2 + X3, 0  ) 
              - calc_fa(X1ym, X1ym + X2, Y3, Y2 + Y3, 0  ) - calc_fa(Y3xm, Y2 + Y3xm, X1, X1 + X2, 0  ) ) ) 
        
    return phiym    

In [19]:
""" 式(25) Test """
# \確認.xlsx "式(25)phiym"シート → \phiym.csv を作成 → \TestConfig01 下に置いて読み込み → 計算

import pandas as pd
import sys
csv_input = pd.read_csv(filepath_or_buffer="./TestConfig01/phiym.csv", encoding="ms932", sep=",")
if csv_input.columns[0]!="phiym_case":
    sys.exit("ファイル内に貼り付けたテスト条件が違います")
for i in range(len(csv_input)):
    [case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, phiymA] \
        = csv_input.values[i]
    phiym = calc_phiym(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym)
    print('case{}: phiym = {}, 期待値 = {}, 残差 = {}'.format( case, phiym, phiymA, phiym - phiymA ))

case1.0: phiym = 0.5, 期待値 = 0.5, 残差 = 0.0
case2.0: phiym = 0.4981403380408517, 期待値 = 0.498140338040852, 残差 = -2.7755575615628914e-16
case3.0: phiym = 0.49268111920450114, 期待値 = 0.4926811192045011, 残差 = 5.551115123125783e-17
case4.0: phiym = 0.47245663589823916, 期待値 = 0.472456635898239, 残差 = 1.6653345369377348e-16
case5.0: phiym = 0.27960145072446174, 期待値 = 0.27960145072446196, 残差 = -2.220446049250313e-16
case6.0: phiym = 0.04702694222229747, 期待値 = 0.0470269422222975, 残差 = -3.469446951953614e-17
case7.0: phiym = 0.5, 期待値 = 0.5, 残差 = 0.0
case8.0: phiym = 0.4482669006200264, 期待値 = 0.448266900620026, 残差 = 3.885780586188048e-16
case9.0: phiym = 0.40352744336645047, 期待値 = 0.40352744336645097, 残差 = -4.996003610813204e-16
case10.0: phiym = 0.32930158326613274, 期待値 = 0.329301583266133, 残差 = -2.7755575615628914e-16
case11.0: phiym = 0.11345861265124076, 期待値 = 0.11345861265124099, 残差 = -2.3592239273284576e-16
case12.0: phiym = 0.01158179050235552, 期待値 = 0.0115817905023555, 残差 = 1.9081958235744878

#### 6.5 b) 式(24)

$$\gamma_{isr,j,y-} = 2 \phi_{j,y-} \qquad (24) $$

In [20]:
""" 式(24) """

def calc_gammaym(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym):
    
    gammaym = 2 * calc_phiym(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym)
         
    return gammaym     